## Imports


In [160]:
import cv2
from typing import Tuple, List
from math import sin, cos, pi
from skimage.draw import line as line_points
import numpy as np
from numpy.linalg import norm

## Constants


In [161]:
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
BLACK = (0, 0, 0)
RAD2DEG = 180 / pi
DEG2RAD = pi / 180
FILL = -1
MAX_RANGE = 1200
SENSOR_ANGLE = 250  # (-125,125)
SENSOR_STEP = 2

## Functions


In [162]:
def convert_from_raduis_to_cartesian(
    x: int, y: int, r: float, theta: float
) -> Tuple[float, float]:
    """
    Convert from raduis and theta to cartesian coordinates
    """
    x2 = int(x + r * cos(theta * DEG2RAD))
    y2 = int(y + r * sin(theta * DEG2RAD))

    return (x2, y2)

## Requirement 1


In [165]:
def req1(image_path: str, x_robot: int, y_robot: int, theta_robot: int):
    # read the image using opencv
    img = cv2.imread(image_path)
    # draw red cricle at the robot position which is (x,y,theta) = (350,185,200)
    cv2.circle(img, (x_robot, y_robot), 10, RED, FILL)
    # draw the robot direction
    cv2.line(
        img,
        (350, 185),
        convert_from_raduis_to_cartesian(350, 185, 10, theta_robot),
        BLUE,
        thickness=2,
    )
    # open the measurements file
    # write the position of the robot in txt file
    file = open("observations.txt", "w")
    file.write(
        f"Robot position: x= {str(x_robot)} , y= {str(y_robot)} , theta= {str(theta_robot)}"
    )

    # write meaeurements in txt file
    file.write("\nObservations: ")
    file.write("\nAngle \t Distance(cm) \t Endpoint")

    # create array to save the distance at each angle
    distances = np.zeros((SENSOR_ANGLE // SENSOR_STEP) + 1)

    # list of the locations of the endpoints at each angle
    endpoints = []
    # loop over the sensor angles
    for ind, angle in enumerate(
        range(
            theta_robot - (SENSOR_ANGLE // SENSOR_STEP),
            theta_robot + (SENSOR_ANGLE // SENSOR_STEP) + 1,
            SENSOR_STEP,
        )
    ):
        # get the end of the ray
        x_max, y_max = convert_from_raduis_to_cartesian(
            x_robot, y_robot, MAX_RANGE // 4, angle
        )
        # get all the points in the line
        points_in_line = line_points(
            x_robot,
            y_robot,
            x_max,
            y_max,
        )
        x_points = points_in_line[0]
        y_points = points_in_line[1]
        # get the final point in the line
        x_final = x_points[0]
        y_final = y_points[0]
        # loop over the points in the line
        for i in range(len(x_points)):
            x = x_points[i]
            y = y_points[i]
            if all(img[y, x] == BLACK):
                break
            else:
                x_final = x
                y_final = y

        endpoint = (x_final, y_final)
        # draw a line from the robot to the final point
        cv2.line(
            img,
            (x_robot, y_robot),
            endpoint,
            GREEN,
            thickness=2,
        )
        # calculate the distance between the robot and the endpoint
        distance = norm(np.array((x_robot, y_robot)) - np.array(endpoint))
        # save the distance in the array
        distances[ind] = min(MAX_RANGE, distance * 4)
        endpoints.append(endpoint)

        # write meaeurements in txt file
        file.write(f"\n {str(angle)} \t {str(distances[ind])} \t {str(endpoint)}")
    # save the image
    cv2.imwrite("req1.jpg", img)
    # close the file
    file.close()
    # show the image
    # cv2.imshow("image", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return distances, np.array(endpoints)

In [166]:
distances, endpoints = req1("./Map.jpg", 350, 185, 200)

## Requirement 2


In [159]:
def req2(image_path: str, actual_measures: List[float], variance: float = 1):
    """
    actual_measures: List[float]: Actual measures of the robot
    """

In [158]:
req2("./Map.jpg", distances)